### Import Necessary Packages

In [3]:
import pandas
import numpy
import os
import torch

### Import Data

In [15]:
DATA = os.getcwd()
DATA

'/Users/lyz9518/Documents/GitHub/dlcv_final_project'

NameError: name 'PATH' is not defined

### Data Collection

### Data Process

### Model Construction